In [1]:
import numpy as np
from math import gamma
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import time
import pandas as pd

import os
os.chdir("C:\\Users\\user\\Downloads\\Thesis_paper_1\\calibration")

np.set_printoptions(suppress=True) # supress scientific notation

import matplotlib.ticker as mtick
import locale
locale.setlocale(locale.LC_NUMERIC, "de_DE")


plt.rcParams['axes.formatter.use_locale'] = True



pd.set_option('display.max_columns', 50) 

In [2]:

import tensorflow as tf
import tensorflow_probability as tfp

import functools

from scipy.optimize import differential_evolution

In [3]:
## parameters



def get_data(year):
    w='data_' + str(year)
    alfa = tf.convert_to_tensor(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,5], dtype='float64').reshape(1,3) )
    phi = tf.convert_to_tensor(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,4], dtype='float64').reshape(1,3) )
    return alfa, phi



def pars(year):
    global alfa, rho, beta, theta, eta, psi, kappa, lamb, gamma1, G, phi, s
    rho = tf.constant(0.19, tf.float64)
    beta = tf.constant(0.231, tf.float64)
    theta = tf.constant(2.0, tf.float64)
    eta = tf.constant(0.103, tf.float64)
    psi = eta**eta*(1.0-eta)**(1.0-eta)
    kappa = tf.divide(1.0, (1.0 - eta))
    lamb = 1 - tf.divide(kappa, theta)
    gamma1 = tf.multiply(eta, kappa)    
    G = tf.constant(gamma(1 - tf.divide(kappa, tf.multiply(theta, (1-rho)) ) ), tf.float64  )      
    alfa, phi = get_data(year)
    s = tf.pow( (1 + tf.divide( (1-eta), ( tf.multiply(beta, phi) ) ) ), -1 )


pars(15)

In [4]:

df = pd.DataFrame()

l = [92, 95, 2, 15]

for i in l:
    w='data_' + str(i)
    k ='alfa_' + str(i)
    df[k] = np.array(pd.read_excel(f'{w}.xlsx').iloc[:,5], dtype='float64').reshape(3)



df

In [5]:
df2 = pd.DataFrame()


for i in l:
    w='data_' + str(i)
    j ='phi_' + str(i)
    df2[j] = np.array(pd.read_excel(f'{w}.xlsx').iloc[:,4], dtype='float64').reshape(3)

    
df2

In [6]:
# taus



def taus(st):
    tau_h = tf.random.uniform(maxval=10, minval=-10, shape=[1, st], dtype='float64')
    x = np.array([tau_h])
    x = tf.convert_to_tensor(x, tf.float64)
    return x
    

In [7]:
def s_test():
    phi = tf.convert_to_tensor(np.array([0.1, 0.2, 0.3]).reshape(1,3) )
    S = tf.pow(s, phi*(theta*lamb+ kappa) )
    #S2 =tf.pow((1 - s), ( (theta*lamb)/(beta*kappa) ) )
    T = tf.multiply(S, 1.0)
    return T
    

In [8]:
def N_if(x1):
    S = tf.pow(s, phi*(theta*lamb+ kappa) )
    S2 =tf.pow((1 - s), ( (theta*lamb)/(beta*kappa) ) )
    H = tf.pow((1 + x1[0]), (theta*lamb + kappa))       
    C = tf.multiply(tf.multiply(tf.pow(psi, theta*lamb), eta**gamma1 ), G )
    N_i = tf.multiply(S, tf.multiply(S2, tf.multiply(H, C)) )
    return N_i

In [9]:

#  linear system 

def matrix():
    global A, B
    A = np.repeat(alfa, st).reshape(st, st).T       # Matriz de parâmetros    
    B = np.repeat(alfa, st).reshape(st, st).T        # Matriz de alfas 
    for i in range(st):
        for j in range(st):
            if i==j:
                A[i, j] = 1 - (A[i, j] - 1)*(theta*lamb + gamma1)
                B[i, j] = B[i, j] - 1
            if i != j:
                A[i, j] = -A[i, j]*(theta*lamb + gamma1) 
    A = tf.convert_to_tensor(A, tf.float64)
    B = tf.convert_to_tensor(B, tf.float64)
    

def lin_sys(x1):
    #global N_i
    #print(f'x1 = {x1}')
    N_i = N_if(x1)
    N_i2 = tf.math.log( N_i )                                                    # Logaritmo de N_i
    C = tf.math.log(alfa)                                                        # Logaritmo dos alfas
    D = tf.reshape(tf.tensordot(B, tf.reshape(N_i2, (3,1)) , 1), (1, 3) ) + C    # produto matricial                     
    res = tf.reshape(tf.exp(tf.linalg.solve(A, tf.reshape(D, (3,1)) ) ), (1,3) )  # Resolvendo o sistema  A w = D
    #res =tf.reshape(res, (1,3))
    return res

In [10]:
def w_til_f(x1):
    S = tf.pow(s, phi)
    S2 = tf.pow( (1 - s), 1/(beta*kappa) )
    H = 1 + x1[0]
    #print(f'x1[0] = {x1[0]}')
    w_til = tf.multiply(psi, tf.multiply(H, tf.multiply(S, tf.multiply(w , S2))))
    return w_til

   

In [11]:
# wages - model 

def W_i_f(x1):
    w_til = w_til_f(x1)
    sum_wtil = tf.pow( tf.reduce_sum( tf.pow(w_til, theta) ) , (kappa/theta) ) 
    S = tf.pow((1-s), (-1/beta))
    p_i = tf.pow(w_til, theta)/ tf.reduce_sum( tf.pow(w_til, theta) )   
    W_i = tf.multiply(kappa, tf.multiply(G, tf.multiply(S, tf.multiply(sum_wtil, p_i)) ) )    
    return W_i, p_i

In [12]:
# human capital

def hc_f(x1):
    W = tf.pow(w , (theta*lamb + gamma1) )
    W_til = tf.pow(tf.reduce_sum(tf.pow(w_til_f(x1), theta) ), lamb)
    N_i = N_if(x1)
    Hc = tf.multiply(N_i, tf.divide(W, W_til))
    return Hc

In [13]:
# PNAD - DATA


def get_p_i(year):
    global p_s2
    w='data_' + str(year)
    p_i = tf.convert_to_tensor(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,2]).reshape(1,3), tf.float64)
    p_s2 = p_i[1:st]
    return p_i


def get_W_i(year):
    p_i = get_p_i(year)
    global W_id
    w='data_' + str(year)
    W_id = tf.convert_to_tensor(np.multiply(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,1]), p_i).reshape(1,3), tf.float64)
    return W_id

In [14]:
#@tf.function
def tflow(x1):    
    global w
    w = lin_sys(x1)
    #print(f'shape = {x1.shape}')
    x1 = tf.reshape(x1, (1, 3))
    #print(f'shape = {x1.shape}')
    #tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    #D = tf.math.log(D)
    return D

In [15]:
st=3
x1 = taus(st)
ano = 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [16]:
st=3
x1 = taus(st)
ano = 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [17]:
st=3
x1 = taus(st)
ano = 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [18]:
# bounds


Bd =   ((-0.9998, 30.0), )*st 

Bd = np.array(Bd, dtype=object)

In [19]:
# callback


cc = 0
def callback(x):
    global cc
    cc += 1
    fobj = tflow(x)
    print(f'\033[1;033mObjetivo: {np.around(fobj, 10)}, iter: {cc}') 

In [20]:
# obter as distorções

def get_dist(ano):
    pars(ano)
    matrix()
    return np.array(pd.read_excel('dist.xlsx').loc[:, f'dist_{ano}'], dtype='float64').reshape(3)

In [21]:
def info(ano):
    d= get_dist(ano).reshape(1,3)
    pars(ano)
    matrix()
    print('\033[1;034m')
    print(15*'*'+ ' OUTPUT '+ f' Ano: {ano} ' + 15*'*')
    print(' ')
    print(f'Distorção =', *np.round(d.flatten(), 4) )
    print(f'Shape da distorção = {d.shape}')  
    print(f'N_i = {N_if(d)}')
    print(f'w_til = {w_til_f(d)}')
    print(f'W_i = {W_i_f(d)[0]}') 
    print(f'W_dados = {get_W_i(ano)}')
    print(f'p_i = {W_i_f(d)[1]}')
    print(f'alfa = {alfa}')
    print(f'phi = {phi}')
    print(f's = {s}')
    #print(f'A = {A}')
    #print(f'B = {B}')
    obj=tflow(get_dist(ano)) 
    print(f'Objetivo = {obj}')
    print(f'w = {w}')
    print(f'capital humano = {hc_f(d)}')
    print(' ')
    print(47*'*')

In [22]:
def ajuste(ano):
    info(ano)
    d= get_dist(ano).reshape(1,3)
    pars(ano)
    matrix()
    tflow(get_dist(ano))

    W_m = W_i_f(d)[0]
    W_d = get_W_i(ano)
    
    p_m = W_i_f(d)[1]
    p_d = get_p_i(ano)
    
    fig,ax = plt.subplots(1, 2, figsize=(14,8))
    ax[0].scatter(p_m, p_d, s=100, color='red')
    ax[0].plot([0.0, 0.7], [0.0, 0.7], 'k-', lw=2, label='linha de 45°', color='blue')
    ax[0].grid(True)
    ax[0].set_title(r'$p_i$', fontsize=20)
    ax[0].tick_params(axis='both', which='major', labelsize=20)

    ax[1].scatter(W_m, W_d, s=100, color='red')
    ax[1].plot([np.min(W_m), np.max(W_m)], [np.min(W_m), np.max(W_m)], 'k-', lw=2, label='linha de 45°', color='blue')
    ax[1].grid(True)
    ax[1].set_title(r'$W_i$', fontsize=20)
    ax[1].tick_params(axis='both', which='major', labelsize=20)


    
    

In [23]:
ajuste(15)

In [24]:

l = [92, 95, 2, 15]


df3 = pd.DataFrame()

for i in l:
    d = get_dist(i).reshape(1,3)
    pars(i)
    matrix()
    obj = tflow(get_dist(i)) 
    #w='data_' + str(i)
    k ='hc_' + str(i)
    df3[k] = np.array(hc_f(d)).reshape(3)


df3.index = ['AGR', 'IND', 'SERV']

In [25]:
df3

In [26]:
anos = [92, 95, '02', 15]
up = 0.04

fig, ax = plt.subplots(2, 2, figsize=(16,16))
x = [ax[0,0], ax[0,1], ax[1,0], ax[1,1]]

for i in range(len(x)):
    a = df3.iloc[:,i].keys()
    b = df3.iloc[:,i].values
    x[i].bar(a, b, color ='cyan', width = 0.2)
    x[i].set_title('Capital Humano'+ f' {anos[i]}', fontsize=20)
    x[i].tick_params(axis='both', which='major', labelsize=20)
    x[i].grid(b=True, which='major', color='k', linestyle=':', lw=.5, zorder=1)
    for xi, yi, l in zip(*[a, b, list(map(str, np.round(b,2)))] ):
        x[i].text(xi, yi + up, l,
                  horizontalalignment='center',
                #bbox=dict(facecolor='w', edgecolor='w', alpha=.5),
               fontsize=16)
    plt.tight_layout

    

In [27]:
# pegar os anos de estudo da pnad

def get_est(year):
    w='data_' + str(year)
    est = np.array(pd.read_excel(f'{w}.xlsx').iloc[:,3], dtype='float64').reshape(3) 
    return est

In [28]:
# pegar o capital humano do modelo que está em df3

def calc_hc(year):
    n = 'hc_' + str(year)
    return np.array(df3.loc[:, [n]]).reshape(3)

In [29]:
# cada ponto de dispersão é um dentre os quatro anos analisados
# cada gráfico é de um setor

sector = ['Agropecuária', 'Indústria', 'Serviços']
anos = [92, 95, 2, 15]
n=len(anos)    # quantidade de anos
fig, ax = plt.subplots(3, 1, figsize=(8,16))

for j in range(3):
    ax[j].scatter([[calc_hc(i) for i in anos][k][j] for k in range(n) ] ,
                  [[get_est(i) for i in anos][k][j] for k in range(n) ] , marker='o', s=200, color='maroon' )
    ax[j].tick_params(axis='both', which='major', labelsize=20)
    ax[j].set_xlabel('Capital humano - Modelo', fontsize=20)
    ax[j].set_ylabel('Anos de estudo - PNAD', fontsize=20)
    ax[j].grid(b=True, which='major', color='k', linestyle=':', lw=.5)
    ax[j].set_title(f'{sector[j]}', fontsize=20)
plt.tight_layout()

In [30]:
# calcula o produto da economia

def pib(year):
    Y = tf.math.reduce_prod(tf.pow(calc_hc(year), get_data(year)[0] ))
    return Y

    

In [31]:
# pega o pib dos dados do ipea

def pib_dados(year):
    da = pd.read_excel('IPEA_PIBS.xls', sheet_name='s2').iloc[:,[9,10]]
    da.set_index('index', inplace=True)
    return np.array(np.log(da.loc[[year]] ) ).reshape(1)

In [32]:
# cada ponto de dispersão é um ano diferente
# aqui uso o loop dentro da lista

anos = [92, 95, 2, 15]


fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter([pib_dados(i) for i in anos] , 
            [pib(i) for i in anos], marker='o', s=200, color='maroon' )
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Logaritmo do PIB - Dados', fontsize=20)
ax.set_ylabel('PIB - Modelo', fontsize=20)

In [33]:
anos = [92, 95, 2, 15]
sector = ['Agropecuária', 'Indústria', 'Serviços']
up = 0.01


fig, ax = plt.subplots(2, 2, figsize=(16,16))
x = [ax[0,0], ax[0,1], ax[1,0], ax[1,1]]

#d= get_dist(anos[i]).reshape(1,3)


for i in range(len(anos)):
    pars(anos[i])
    matrix()
    tflow(get_dist(anos[i]))
    b = tf.reshape(w, 3)
    x[i].bar(sector, b, color ='gold', width = 0.2)
    x[i].set_title('Salário'+ f' {anos[i]}', fontsize=20)
    x[i].tick_params(axis='both', which='major', labelsize=20)
    x[i].grid(b=True, which='major', color='k', linestyle=':', lw=.5, zorder=1)
    for xi, yi, l in zip(*[sector, b, list(map(str, np.round(b,3)))] ):
        x[i].text(xi, yi+up, l,
                #bbox=dict(facecolor='w', edgecolor='w', alpha=.8),
                  horizontalalignment='center',
               fontsize=16)
    plt.tight_layout

In [34]:
df3.plot()

In [35]:
df3.plot(type='bar')

In [36]:
df3.plot(type=bar)

In [37]:
df3.plot(type='bar')

In [38]:
df3.plot('bar')

In [39]:
df3.bar()

In [40]:
df3.plot(kind='bar')

In [41]:
df3.plot(kind='bar', grid=True, figsize=(10,8))

In [42]:

ibre = pd.read_excel(r'C:\Users\user\Downloads\Thesis_paper_1\dados\prod_trab.xlsx', sheet_name='prod_hr')

In [43]:
ibre.index= ibre['Anos']

In [44]:
ibre2 = ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].T

In [45]:
ibre2.index = ['AGR', 'IND', 'SERV']

ibre2.columns = ['95', '2', '15']

In [46]:
prod = df3.iloc[:, -3:]

In [47]:
P_D = np.log(np.array(ibre2).reshape(9) )     # dados logaritmizado
P_M = np.log(np.array(prod).reshape(9) )      # modelo 

In [48]:
anos2 =  [95, 2, 15]
agr = ['AGR_' + str(i) for i in anos2 ]
ind = ['IND_' + str(i) for i in anos2 ]
serv = ['SERV_' + str(i) for i in anos2 ]

In [49]:
texto = np.array([agr, ind, serv], dtype=object).reshape(9)

In [50]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i]+0.05, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18})

In [51]:
prod2 = pd.read_excel('tab-6.xlsx', sheet_name='dados')
prod2.index = prod2['anos']
prod2 = prod2.iloc[:,-2:]

In [52]:
prod2.plot(figsize=(10,8), grid=True)

In [53]:
# wages - model 

def W_med(x1):
    w_til = w_til_f(x1)
    sum_wtil = tf.pow( tf.reduce_sum( tf.pow(w_til, theta) ) , (kappa/theta) ) 
    S = tf.pow((1-s), (-1/beta))
    W_i = tf.multiply(kappa, tf.multiply(G, tf.multiply(S, tf.multiply(sum_wtil, 1.0)) ) )    
    return W_i

In [54]:
l = [92, 95, 2, 15]


df5 = pd.DataFrame()

for i in l:
    d = get_dist(i).reshape(1,3)
    #obj = tflow(get_dist(i)) 
    k ='W_med_' + str(i)
    df5[k] = np.array(W_med(d)).reshape(3)


df5.index = ['AGR', 'IND', 'SERV']

In [55]:
df5.plot(figsize=(10,8), grid=True,  kind='bar', fontsize=18)

In [56]:
W_med = np.log(np.array(df4.iloc[:,-3:]).reshape(9) )

In [57]:
W_med = np.log(np.array(df5.iloc[:,-3:]).reshape(9) )

In [58]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, W_med, s=200, color='maroon')
for i in range(len(texto)):
    ax.text(P_D[i]+0.05, W_med[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Salário Médio - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18});

In [59]:
df5 = pd.DataFrame()

anos = [92, 95, 2, 15]


for i in anos:    
    tflow(get_dist(anos[i]))    
    obj = tflow(get_dist(i)) 
    #w='data_' + str(i)
    k ='w_' + str(i)
    df5[k] = np.array(w).reshape(3)

In [60]:
df5 = pd.DataFrame()

anos = [92, 95, 2, 15]


for i in anos:    
    tflow(get_dist(i)) 
    k ='w_' + str(i)
    df5[k] = np.array(w).reshape(3)

In [61]:
df5

In [62]:
df5.index

In [63]:
df5.index()

In [64]:
df5.index

In [65]:
df5.index = ['AGR', 'IND', 'SERV']

In [66]:
df5

In [67]:
df5.plot(kind='barh')

In [68]:
df5.plot(kind='bar')

In [69]:
df5.plot(kind='bar', grid=True, figsize=(10,8))

In [70]:
prod2.plot(figsize=(10,8), grid=True, lwd=2)

In [71]:
prod2.plot(figsize=(10,8), grid=True, lw=2)

In [72]:
prod2.plot(figsize=(10,8), grid=True, lw=3)

In [73]:
prod2.plot(figsize=(10,8), grid=True, lw=3, fontsize=18)

In [74]:
prod2.plot(figsize=(10,8), grid=True, lw=3, fontsize=18, size=18)

In [75]:
prod2.plot(figsize=(10,8), grid=True, lw=3, fontsize=18)

In [76]:
df5

In [77]:
df5.iloc[:,-3:]

In [78]:
prod_marg = df5.iloc[:,-3:]

In [79]:
prod_marg = np.array(df5.iloc[:,-3:]).reshape(9)

In [80]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, W_med, s=200, color='maroon')
for i in range(len(texto)):
    ax.text(P_D[i]+0.05, prod_marg[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produto Marginal - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18});

In [81]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, prod_marg, s=200, color='maroon')
for i in range(len(texto)):
    ax.text(P_D[i]+0.05, prod_marg[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produto Marginal - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18});

In [82]:
prod2 = pd.read_excel('tab-6.xlsx', sheet_name='dados')
prod2.index = prod2['anos']
prod2 = prod2.iloc[:,-2:]

In [83]:
prod2.plot(figsize=(10,8), grid=True, lw=3, fontsize=18)

In [84]:
ibre.plot(kind='bar')

In [85]:
ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].T.plot(kind='bar')

In [86]:
ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].plot(kind='bar')

In [87]:
ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].T.plot(kind='bar')

In [88]:
ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].T.plot(kind='bar', figsize=(10,8), grid=True)

In [89]:
ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].T.plot(kind='bar', 
                                                                                           figsize=(10,8), 
                                                                                           grid=True,
                                                                                          fontsize=18)

In [90]:
ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].T.plot(kind='bar', 
                                                                                           figsize=(10,8), 
                                                                                           grid=True,
                                                                                          fontsize=18, 
                                                                                          title= Produtividade - IBRE)

In [91]:
ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].T.plot(kind='bar', 
                                                                                           figsize=(10,8), 
                                                                                           grid=True,
                                                                                          fontsize=18, 
                                                                                          title= 'Produtividade - IBRE')

In [92]:
ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].T.plot(kind='bar', 
                                                                                           figsize=(10,8), 
                                                                                           grid=True,
                                                                                          fontsize=18, 
                                                                                          title= 'Produtividade - IBRE');

In [93]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i]+0.05, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18});

In [94]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='cyan')
for i in range(len(texto)):
    ax.text(P_D[i]+0.05, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18});

In [95]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='gold')
for i in range(len(texto)):
    ax.text(P_D[i]+0.05, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18});

In [96]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='maroon')
for i in range(len(texto)):
    ax.text(P_D[i]+0.05, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18});

In [97]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, prod_marg, s=200, color='maroon')
for i in range(len(texto)):
    ax.text(P_D[i]+0.05, prod_marg[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produto Marginal - Modelo', fontsize=20)
ax.title('Produto marginal e produtividade IBRE')
(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18});

In [98]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, prod_marg, s=200, color='maroon')
for i in range(len(texto)):
    ax.text(P_D[i]+0.05, prod_marg[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produto Marginal - Modelo', fontsize=20)
ax.set_title('Produto marginal e produtividade IBRE')
(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18});

In [99]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, prod_marg, s=200, color='maroon')
for i in range(len(texto)):
    ax.text(P_D[i]+0.05, prod_marg[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produto Marginal - Modelo', fontsize=20)
ax.set_title('Produto marginal e produtividade IBRE', fontsize=20)
(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18});

In [100]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, prod_marg, s=100, color='maroon')
for i in range(len(texto)):
    ax.text(P_D[i]+0.05, prod_marg[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produto Marginal - Modelo', fontsize=20)
ax.set_title('Produto marginal e produtividade IBRE', fontsize=20)
(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18});

In [101]:
df5.plot(kind='bar', grid=True, figsize=(10,8), fontsize=18)

In [102]:
df5.plot(kind='bar', grid=True, figsize=(10,8), fontsize=20)

In [103]:
df5.plot(figsize=(10,8), grid=True,  kind='bar', fontsize=18);

In [104]:
df3.T.plot(kind='bar', grid=True, figsize=(10,8))

In [105]:
df3.plot(kind='bar', grid=True, figsize=(10,8))

In [106]:
df3.plot(kind='bar', grid=True, figsize=(10,8), fontsize=20)

In [107]:
pd.read_excel('dist.xlsx')

In [108]:
pd.read_excel('dist.xlsx').iloc[:,-4:]

In [109]:
pd.read_excel('dist.xlsx').iloc[:,-4:].plot()

In [110]:
pd.read_excel('dist.xlsx').iloc[:,-4:].plot(kind='bar')

In [111]:
D = pd.read_excel('dist.xlsx').iloc[:,-4:]

In [112]:
D = pd.read_excel('dist.xlsx').iloc[:,-4:]
D.index = = ['AGR', 'IND', 'SERV']
D.plot(kind='bar', figsize=(10,8), fontsize=20, grid=True)

In [113]:
D = pd.read_excel('dist.xlsx').iloc[:,-4:]
D.index = ['AGR', 'IND', 'SERV']
D.plot(kind='bar', figsize=(10,8), fontsize=20, grid=True)

In [114]:
D = pd.read_excel('dist.xlsx').iloc[:,-4:]
D.index = ['AGR', 'IND', 'SERV']
D.T.plot(kind='bar', figsize=(10,8), fontsize=20, grid=True)